MNIST(10) augmentation -> CIFAR (5/epocaMNIST)
xresnet34
fit normal
20% do dataset

In [1]:
from fastai.vision.all import *
from datetime import datetime
import torch
torch.cuda.is_available()

True

In [2]:
path = Path('/home/fredguth/.fastai/data/cifar10_mnist')

In [3]:
def label_func(f):
    return (str(f).split("_y")[1:][0][0])

def noise_func(f):
    return (str(f).split("_n")[1:][0][0])

In [4]:
def get_dls(task="CIFAR", size=64):
    dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   splitter  = RandomSubsetSplitter(train_sz=0.16, valid_sz=0.04, seed=42),
                   get_items = get_image_files,
                   get_y = label_func if (task == 'CIFAR') else noise_func,
                   batch_tfms= [*aug_transforms(size=size, min_scale=0.75), Normalize],
                   n_inp     = 1 )
    return dblock.dataloaders(path, num_workers=4)

In [5]:
class HeadBodyNetwork(Module):
    def __init__(self, head, body):
        self.head = head
        self. body = body
    def forward(self, x):
        return self.head(self.body(x))

In [6]:
def headbody_splitter(m): return L(m.head, m.body).map(params)

In [7]:
class RunMNIST(Callback):
    def after_epoch(self):  
        epoch = self.epoch
        backbone = copy(self.learn.model[0])
        head =  create_head(512,10)
        net = HeadBodyNetwork(head=head, body=backbone)
        l =Learner(
                           dls=get_dls(task="CIFAR"), 
                           model=net,
                           splitter=headbody_splitter,
                           loss_func=F.cross_entropy, 
                           metrics=accuracy, 
                           cbs=[CSVLogger(fname=f"results/ex4/cifar_after_mnist_e{epoch}.csv")])
        l.freeze()
        l.fit(3,0.002)

In [8]:
mnist_learner= cnn_learner(dls=get_dls(task="MNIST"), 
                           arch=xresnet18, 
                           pretrained=False, 
                           loss_func=F.cross_entropy, 
                           metrics=accuracy, 
                           cbs=[CSVLogger(fname=f"results/ex4/mnist.csv"), RunMNIST]
                          )

In [9]:
mnist_learner.fit(10,0.005)

epoch,train_loss,valid_loss,accuracy,time
0,2.518268,2.220341,0.322500,00:25
1,1.394318,1.066510,0.690000,00:24
2,0.887635,0.866399,0.755000,00:25
3,0.608814,0.589274,0.839500,00:26
4,0.513494,0.195466,0.936000,00:27
5,0.449255,0.261680,0.918500,00:29
6,0.389956,0.405958,0.880000,00:31
7,0.388431,0.226787,0.927500,00:34
8,0.338783,0.433201,0.878500,00:39
9,0.295050,0.178998,0.944000,00:38


epoch,train_loss,valid_loss,accuracy,time
0,2.036136,1.823364,0.314500,00:05
1,1.842595,1.867399,0.351000,00:05
2,1.702046,1.681541,0.395000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,2.068151,1.771725,0.363000,00:05
1,1.804077,1.791673,0.389000,00:05
2,1.655864,1.525093,0.460000,00:05


epoch,train_loss,valid_loss,accuracy,time
0,1.786472,1.506314,0.462500,00:05
1,1.570783,1.449006,0.480000,00:06
2,1.460028,1.415671,0.502000,00:05


epoch,train_loss,valid_loss,accuracy,time
0,1.641883,1.522166,0.474000,00:06
1,1.422158,1.523100,0.499000,00:06
2,1.338886,1.249744,0.577500,00:06


...

epoch,train_loss,valid_loss,accuracy,time
0,1.450578,1.398556,0.532000,00:07
1,1.242702,1.216143,0.588000,00:07
2,1.159916,1.064543,0.628500,00:07


epoch,train_loss,valid_loss,accuracy,time
0,1.426542,1.303872,0.547500,00:07
1,1.214587,1.168301,0.603500,00:07
2,1.112853,1.164938,0.610000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,1.294703,1.280630,0.561000,00:07
1,1.118780,1.147748,0.612000,00:08
2,1.058489,1.117367,0.639000,00:09


epoch,train_loss,valid_loss,accuracy,time
0,1.235379,1.186228,0.596500,00:09
1,1.059600,1.125797,0.633500,00:09
2,1.005679,1.142999,0.628000,00:09


epoch,train_loss,valid_loss,accuracy,time
0,1.239139,1.177083,0.611000,00:09
1,1.013010,1.127811,0.622500,00:09
2,0.930753,1.058599,0.637500,00:09


In [10]:
import pandas as pd
import glob

In [11]:
!pwd

/home/fredguth/unb/cifar_mnist


In [12]:
all_files = glob.glob("/home/fredguth/unb/cifar_mnist/results/ex4/cifar*.csv")

In [13]:
all_files

['/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e8.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e0.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e2.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e3.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e1.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e5.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e7.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e6.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e4.csv',
 '/home/fredguth/unb/cifar_mnist/results/ex4/cifar_after_mnist_e9.csv']

In [14]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [15]:
frame = pd.concat(li, axis=0, ignore_index=True)

In [16]:
mnist_results = frame.loc[frame['epoch'] == 2]

In [17]:
mnist_results

,epoch,train_loss,valid_loss,accuracy,time
2,2,1.005679,1.142999,0.6280,00:09
5,2,1.702046,1.681541,0.3950,00:06
8,2,1.460028,1.415671,0.5020,00:05
11,2,1.338886,1.249744,0.5775,00:06
14,2,1.655864,1.525093,0.4600,00:05
17,2,1.159916,1.064543,0.6285,00:07
20,2,1.058489,1.117367,0.6390,00:09
23,2,1.112853,1.164938,0.6100,00:07
26,2,1.223617,1.209366,0.5795,00:06
29,2,0.930753,1.058599,0.6375,00:09


In [18]:
mnist_results = mnist_results.reset_index().drop(columns=['index', 'epoch'])

In [19]:
mnist_results.to_excel("results/ex4/ex4_cifar_results.xlsx")

In [20]:
df = pd.read_csv("/home/fredguth/unb/cifar_mnist/results/ex4/mnist.csv", index_col=None, header=0)

In [22]:
df.to_excel("/home/fredguth/unb/cifar_mnist/results/ex4/mnist.xlsx")